# Sample Database Code

Steal this code.

The TIdatabase module encapsulates all of the loading, storing and joining of the student, college and applications.

The module should be imported at the beginning of your code, whether it is a regular Python file or an iPython Notebook.

In [1]:
import TIdatabase as ti

In [2]:

students = ti.Student()
# populate students with random values
students.fillRandom(10)
students.df 

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,canAfford,GPA,female,MinorityGender,...,outofstate,international,firstinfamily,alumni,sports,artist,program,workexp,schooltype,intendedgradyear
0,J12UW1759C,0.395545,0.871726,0.797346,0.204558,0.686297,-1,0.587644,1,-1,...,-1,1,-1,-1,0,0,1,1,1,2011
1,06E1Z0N5UE,0.295840,0.485177,0.215723,0.590468,0.643808,0,0.433858,0,0,...,0,0,0,1,1,0,5,-1,4,2010
2,GACLAHL7LF,0.570530,0.701138,0.276469,0.292684,0.291785,1,0.782334,1,-1,...,0,0,1,-1,1,-1,3,0,5,2010
3,H8SVA3AYRQ,0.447136,0.436797,0.162559,0.297325,0.762663,0,0.570070,1,1,...,1,0,-1,-1,-1,0,1,1,3,2011
4,UDQA4QXITN,0.001344,0.135860,0.694113,0.634774,0.135874,-1,0.206995,1,-1,...,-1,0,-1,-1,0,-1,4,0,1,2014
5,GOB7OM9UCY,0.483627,0.958416,0.118517,0.909398,0.575826,1,0.393563,-1,-1,...,-1,1,-1,0,0,0,5,1,1,2010
6,BFBC9XT25A,0.987455,0.283389,0.822701,0.007015,0.259213,1,0.579281,0,-1,...,-1,1,1,1,1,-1,1,-1,2,2012
7,NPOMGSAIMP,0.896073,0.714154,0.439229,0.639153,0.326818,0,0.300716,-1,0,...,1,-1,0,0,-1,-1,2,0,2,2015
8,XJJ7SRTJ2O,0.676246,0.198814,0.813057,0.088937,0.475929,-1,0.594958,1,-1,...,0,1,-1,1,0,1,4,-1,5,2017
9,JPRVSKBK50,0.603240,0.733572,0.455183,0.131928,0.364909,-1,0.078695,-1,-1,...,0,-1,1,1,0,0,5,1,1,2016


The list of colleges is populated from a CSV stored in the same directory as this notebook. It can be edited using your favorite CSV editor, such as Excel. When you create a new instance of College, the values will be read in from the CSV

In [3]:
# populate with list of known colleges
colleges = ti.College()
colleges.df

,name,acceptrate,size,public,finAidPct,instatePct
0,Harvard,0.05,8000,-1,0.5,0.2
1,Yale,0.06,7000,-1,0.5,0.1
2,Umass,0.80,12000,1,0.8,0.9


In [ ]:
This is the table of application forms

In [4]:
applForm = ti.ApplForm()
applForm.fillRandom(30)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,GOB7OM9UCY,Yale,-1,-1,1,0
1,J12UW1759C,Harvard,-1,1,1,0
2,UDQA4QXITN,Harvard,-1,-1,1,0
3,JPRVSKBK50,Umass,-1,0,1,0
4,H8SVA3AYRQ,Yale,0,-1,0,0
5,XJJ7SRTJ2O,Yale,0,1,0,0
6,BFBC9XT25A,Umass,-1,0,-1,0
7,JPRVSKBK50,Harvard,0,-1,-1,0
8,XJJ7SRTJ2O,Harvard,0,-1,1,0
9,NPOMGSAIMP,Yale,1,-1,-1,0
